# Data Management Exercise using ISA Model and ISA API 

This notebook is an exercise relying on the ISA-API functionality for building an experiment description based on the experimental design.

### Objectives
- Identify concepts from the experimental design
- Represent the experimental design concepts using the ISA-API model  
- Produce ISA metadata for the experimen

### Authors and Contact:
- http://isa-tools.org/team/
- [get in touch](mailto:isatools@googlegroups.com)



### References
- [Documentation about the ISA-API](https://isatools.readthedocs.io), and more specifically:
    - [documentation about study-design driven creation of ISA content](https://isatools.readthedocs.io/en/latest/studydesigncreation.html)
    - [documentation of main ISA objects](https://isatools.readthedocs.io/en/latest/creation.html#)


# Case Study 1: A Toxicity Study in Rats

Read the following description of tan experiment and follow the steps below to identify the main elements from the experimental design.


Experiment Narrative:
---------------------

*Male Fisher F344 rats* purchased from Charles River were treated with *3 commonly used painkillers*, namely *acetylsalicilic acid, acetaminophen and ibuprofen*, at *2 distinct dose levels*, delivered *per os*.

Equal number of animals (n=5) were allocated to each of the group defined by a *compound, dose level and duration post exposure* (of 1 hour, 2 hours and 4 hours) combination.

Following sacrifice performed by cervical dislocation and exsanguination preceded by anesthesia (ketamine and xylazine solution), *blood and kidney specimens were collected*.
Total RNA were extracted from kidney samples and mRNA sequencing (*transcription profiling*) was performed using paired-end libraries on Illumina sequencing platform using an *Illumina HiSeq 2000 instrument*.

*Blood samples* were collected at sacrifice time and immediately placed in precooled 60 percent methanol ammonium bicarbonate buffer to quench cellular metabolism. Blood Metabolites were separated in water-soluble and lipophilic fractions. 
*Metabolite profiling* was performed on the polar metabolite fraction only, using *flow injection analysis (FIA) mass spectrometry* on an *Agilent 6550 iFunnel Q-TOF Mass Sprectrometry platform*. Each fraction was injected twice and data were acquired in both ionization modes (positive mode and negative mode).
Raw data files were saved in native instrument format and later converted to HUPO-PSI standard format for mass spectrometry.


## Structuring experimental description with ISA metadata model

In order to describe the experiment, we rely on the models defined by the ISA-API. We thus need the following import statements to make the relevant functions available to our environment:

In [1]:
from isatools.create.models import *
from isatools.model import *

## Identification of Variables, their levels and definition of the Treatment Plans

In the context of the above experiment, ** can you identify the independent variables and their associated levels**?

Once done, declare the corresponding ISA objects. To do so, one may define a new variable by relying in the ```StudyFactor``` object in this way:

```python
chemical_agent = StudyFactor(name="chemical agent")

```
One may also perform some semantic markup. To do so, you may add an ontology term for such study factor, for instance using the [EBI Ontology Lookup Service](https://www.ebi.ac.uk/ols/) to find a relevant term from the ChEBI ontology, and build the factor in this way:
    
```python
chemical_agent = StudyFactor(name="agent", factor_type=OntologyAnnotation(term="chemical entity", term_source="ChEBI", term_accession="http://purl.obolibrary.org/obo/CHEBI_24431"))
```    

Define the relevant ```StudyFactor```'s below and see if you can find ontology terms that are relevant to annotate them (e.g. you can try and find terms from the Experimental Factor Ontology or EFO):


In [2]:
### HERE YOUR ANSWER

chemical_agent = StudyFactor(name="agent", 
                             factor_type=OntologyAnnotation(
                                    term="chemical entity", 
                                    term_source="ChEBI", 
                                    term_accession="http://purl.obolibrary.org/obo/CHEBI_24431"))

Next, we can use a ```TreatmentFactory``` call to include the ```StudyFactor``` and its associated factor levels, i.e. the values defined by the experimentalist and that the variable will assume over the course of experiment execution.

You should create the treatment factory providing the different study factors:

```python
treatment_factory = TreatmentFactory(factors=[ ... here list the variables...])

```
and then for each factor, you can add their levels in the following way:

```python
treatment_factory.add_factor_value( ... factor variable ..., { ... list of strings with the names of the factor values... }
```


In [3]:
### HERE YOUR ANSWER

treatment_factory = TreatmentFactory(factors=[chemical_agent])
treatment_factory.add_factor_value(chemical_agent, {'value 1', 'etc.'})

### Computing the Number of Unique Treatment Groups/Study Groups: 


The experiment plan follows a full factorial design where all combinations of factor values are considered. We can obtain the different treatments by relying on a utility method that, given the factors and their levels, computes the actual set of possible treatments:


In [4]:
all_treatments = treatment_factory.compute_full_factorial_design()

How many treatment groups have been computed? Check the number you considered matches the output of running the command below:

In [5]:
print('Number of study groups (treatment groups): {}'.format(len(all_treatments)))

Number of study groups (treatment groups): 2


We can now build a treatment plan, or ```TreatmentSequence```, by including all the treatments according to the factorial design:

In [6]:
treatment_sequence = TreatmentSequence(ranked_treatments=all_treatments)

Are study subjects exposed to a single intervention or to multiple intervention?
  


You can now output a summary of the treatment plan that you created with the following command:

In [7]:
report = make_summary_from_treatment_sequence(treatment_sequence)
report

{'full_factorial': True,
 'length_of_treatment_sequence': 1,
 'list_of_treatments': [[{'factor': 'agent', 'value': 'value 1'}],
  [{'factor': 'agent', 'value': 'etc.'}]],
 'number_of_factor_levels_per_factor': {'agent': ['etc.', 'value 1']},
 'number_of_factors': 1,
 'number_of_treatment': 2,
 'number_of_treatments': 2}

Is the treatment plan report in agreement with the experimental design?

### Study group size

The following code builds a slider (relying on the ```ipywidgets``` library) for you to set the group size, please select the appropriate number according to the experiment description, where group size is the number of subjects per each combination of factor values or treatment group:

In [8]:
from ipywidgets import IntSlider
group_size = IntSlider(value=1, min=0, max=100, step=1, description='Group size:', disabled=False, continuous_update=False, orientation='horizontal', readout=True, readout_format='d')
group_size


The group size value you chose, and that is going to be used in the next section, is:

In [9]:
group_size.value

1

## Sample collection and assay plans

Given the group size selected above, we are now going to build the sample collection and assay plans based on the group size. Given this, can you say if the design is balanced or unbalanced?

In [10]:
plan = SampleAssayPlan(group_size=group_size.value)

Let's now build a ```dictionary``` with the sample collection plan: it should contain key:value pairs with the specimen or sample type as key and the number of samples collected for each type over the course of the study. Here is the code snippet that you should complete:

```python
sample_collection_plan = { "sample type 1": 2, "sample type 2": 1 }
```


In [11]:
### HERE YOUR ANSWER

sample_collection_plan = { "sample type 1": 2, "sample type 2": 1 }


Next, the following code will take the sample_collection_plan object that you built and include all the details in the sample_collection_plan object:

In [12]:
for sample_type in sample_collection_plan:    
    plan.add_sample_type(sample_type)
    plan.add_sample_plan_record(sample_type,sample_collection_plan[sample_type])


### View the sample plan information as a JSON document

This section is meant to show how to serialize key study design descriptors in a compact document serialized in format format. 
Why is this relevant? How would you use this feature? List 3 possible uses.

In [13]:
import json
from isatools.create.models import SampleAssayPlanEncoder
print(json.dumps(plan, cls=SampleAssayPlanEncoder, sort_keys=True, indent=4, separators=(',', ': ')))

{
    "assay_plan": [],
    "assay_types": [],
    "group_size": 1,
    "sample_plan": [
        {
            "sample_type": "sample type 1",
            "sampling_size": 2
        },
        {
            "sample_type": "sample type 2",
            "sampling_size": 1
        }
    ],
    "sample_qc_plan": [],
    "sample_types": [
        "sample type 1",
        "sample type 2"
    ]
}


## Create an ISA experimental description based on the study design and the sampling plan:

In the following section, the task is to build ISA objects relying on the study design information we built above.


Let's first create an ```Investigation``` object to hold all the information about the experiment with this command, which also assigns an identifier for the investigation.

In [14]:
isa_investigation = Investigation(identifier='inv-dtp-exercise')

Now, let's create a study object using the sample and assay plan as well as the ```TreatmentSequence``` object we built before. For this, we will need an object of the ```IsaModelObjectFactory``` class provided in the ISA-API:

In [15]:
isa_object_factory = IsaModelObjectFactory(plan, treatment_sequence)
isa_study = isa_object_factory.create_study_from_plan()

Now, we can link the study to the investigation we created before:

In [16]:
isa_investigation.studies = [isa_study]


and set a name for the study file, considering that to follow the ISA specification, it must start with ```s_``` and have extension ```.txt``` and could have any string in between such that it makes a valid file name. Thus the patter to follow is: ```s_<some string>.txt```.

In [17]:
### HERE YOUR ANSWER

isa_study.filename = '... complete here a filename for the study file...'

Let's now check the study sample table:

In [20]:
from isatools.isatab import dump_tables_to_dataframes as dumpdf
from qgrid import show_grid
import pandas as pd


dataframes = dumpdf(isa_investigation)
sample_table = pd.DataFrame()
try:
    sample_table = next(iter(dataframes.values()))
    
except StopIteration:
    print("Need more details to print table")
show_grid(sample_table)

NameError: name 'pd' is not defined

In [ ]:
try:
    print('Total rows generated: {}'.format(len(sample_table)))
except NameError:
    print("Need more details to print result")

### Study description and study design type


Can you set the study description (or abstract) relying in this code snippet?

```python
isa_study.description = "... here the text of the description..."
```


In [ ]:
### HERE YOUR ANSWER

isa_study.description = "... here the text of the description..."


Next, we would like to specify the type of study design (and we can set multiple values if necessary). 

For this, we will build ontology annotations, as we did for the ```StudyFactor`` objects:

```python
descriptor_1 = OntologyAnnotation(
                  term="... here the label of the term...", 
                  term_source="... here the name of the ontology the term comes from...", 
                  term_accession="... here the URL of the term ...")
```

To determine some of the study design descriptors, consider the following questions and use the Ontology Lookup Service to find relevant terms:

- is the experiment following an 'intervention design' or an 'observation design'?
- is the design 'factorial' or a 'randomized block' design?
- is the design 'full' or 'fractional'?

In [ ]:
### HERE DEFINE YOUR DESCRIPTORS

After you defined the descriptors, you can append them to isa_study.design_descriptors list as follows:

```python
isa_study.design_descriptors.append(descriptor_1)
```


In [ ]:
### HERE APPEND ALL THE DESCRIPTORS YOU DEFINED ABOVE

## Assay and Data Acquisition Plans:

From the textual description of the experiment, identify the *response variables* (a.k.a. dependent variables).

The ISA model defines an *Assay Type* by a pair of descriptors:  a *type of measurement*  and the *technology* used to obtain these measurements.

In the ISA model, there is a series of configuration files that define the vetted values for Measurement Type and Technology type.

An ISA configuration can be accessed from: https://github.com/ISA-tools/Configuration-Files/tree/master/isaconfig-default_v2015-07-02

Given those lists of configurations, let's define the assay types needed for our experiment.

The way to define an assay type is as follows

```python
assay_type_1= AssayType(measurement_type='...here a supported measurement type...', technology_type='...here a supported technology type...')
```
Define below the assay types that you can identify from the experiment narrative:

In [ ]:
### HERE YOUR ANSWER
assay_type_1= AssayType(measurement_type='...here a supported measurement type...', 
                        technology_type='...here a supported technology type...')

Let's now define a python set for the assay types:

In [ ]:
assay_types = set()

You can add the types you defined above to the set as follows:
    
```python
assay_types.add(assay_type_1)
```

Add all your assay types to the assay_types set below:

In [ ]:
### HERE YOUR ANSWER
assay_types.add(assay_type_1)


Let's now visualise the assay types that you created:

In [ ]:
for x in assay_types:
        print(x.measurement_type.term," using ", x.technology_type.term)


## Assay Specific Descriptors
Each data acquisition modality comes with its own set of parameters. These parameters can be set as to capture the specifics of the underlying experimental workflow, which is then rendered as an ISA graph. This section of the exercise illustrates this process by looking at each ofthe experimental workflows used for sequencing and mass spectrometry applications, as declared in the experimental narrative.


### Dealing with Next Generation Sequencing (NGS) Data Acquisition Plan

Select in the following slider the number of technical replicates in the Next Generation Sequence data acquisition plan:

In [ ]:
ngs_technical_replicates = IntSlider(value=0, min=0, max=5, step=1, description='Technical repeats:', disabled=False, continuous_update=False, orientation='horizontal', readout=True, readout_format='d')
ngs_technical_replicates



Let's now identify the instrument and include it in a set:

In [ ]:
### HERE YOUR ANSWER
instrument = '... here include the instrument name...'

In [ ]:
sequencing_instruments = set()
sequencing_instruments.add(instrument) 

How many different libraries were used for NGS? Set the value in the slider below (and you can change the default value so that you don't need to change the slider every time you re-run the cell):

In [ ]:
ngs_distinct_libraries = IntSlider(value=0, min=0, max=5, step=1, description='Distinct libraries:', disabled=False, continuous_update=False, orientation='horizontal', readout=True, readout_format='d')
ngs_distinct_libraries


The value you selected was:

In [ ]:
ngs_distinct_libraries.value

The NGS parameters that you selected in the previous steps are known in ISA speak as 'topology modifiers'. This name refers to the fact that they affect the shape of the experimental graph being considered, where the graph has material or data entities as nodes and processes as edges.

The following code defines a ```DNASeqAssayTopologyModifiers``` object and adds it to the AssayType object defined earlier on:

In [ ]:
top_mods_seq = DNASeqAssayTopologyModifiers(
    technical_replicates=ngs_technical_replicates.value, 
    instruments=sequencing_instruments,
    distinct_libraries=ngs_distinct_libraries.value)

assay_type_1.topology_modifiers = top_mods_seq

print(assay_type_1.topology_modifiers)

Now that we have all the information about the NGS assay type, we can add it to the plan:

In [ ]:
plan.add_assay_type(assay_type_1)

Finally, for the NGS, we need to identify to what sample type it is applied. Fill in the NGS sample type in the following variable:

In [ ]:
### HERE YOUR ANSWER

ngs_sample_type = '... here include the sample type for which NGS is applied...'

The following code will associate the NGS assay type to the relevant sample type (as per your definition above):

In [ ]:
for sample_type in plan.sample_types:
    if sample_type.value.term == ngs_sample_type:
        print('adding assay for sample_type '+sample_type.value.term)
        plan.add_assay_plan_record(sample_type.value.term, assay_type_1)
        assay_plan = next(iter(plan.assay_plan))
    elif sample_type.value.term != ngs_sample_type:
        print('doing nothing for '+sample_type.value.term)
        

Let's now check the assay plan to see if it matches the experiment narrative:

In [ ]:
try:
    print('Added assay plan: {0} -> {1}/{2}'.format(assay_plan[0].value.term, assay_plan[1].measurement_type.term, assay_plan[1].technology_type.term))
    if len(top_mods_seq.instruments) > 0:
        print('Instruments: {}'.format(list(top_mods_seq.instruments)))
except NameError:
    print("Need more details to print assay plan")

In [ ]:
print(json.dumps(plan, cls=SampleAssayPlanEncoder, sort_keys=True, indent=4, separators=(',', ': ')))

### Dealing with Mass Spectrometry Data Acquisition Plan

Moving on, we'll now capture the information about second assay type, related to mass spectrometry.
We want to identify, when relevant:
- the chromatography instruments
- the mass spectrometry instrument
- the injection mode
- the acquisition modes

Given the definition of the following sets, add the relevant elements according to the experimental description:

In [ ]:
chromatography_instruments = set()
ms_instruments = set()
injection_modes = set()
acquisition_modes = set()


In [ ]:
### HERE YOUR ANSWER

How many technical replicates are used for the mass spectrometry assay? Set the appropriate value below:


In [ ]:
### HERE YOUR ANSWER 
ms_tech_rep = 0

Given all this information, we are in a position to define an object of the class ```MSAssayTopologyModifiers``` and assign it to the corresponding assay type:

In [ ]:
try:
    top_mods_ms = MSAssayTopologyModifiers(
          technical_replicates=ms_tech_rep, 
          injection_modes=injection_modes, 
          acquisition_modes=acquisition_modes, 
          instruments=ms_instruments, 
          chromatography_instruments=chromatography_instruments)

    assay_type_2.topology_modifiers = top_mods_ms
except NameError:
    print("There was an assay type left undefined")

Let's see all the settings to check they are as expected:

In [ ]:
if len(top_mods_ms.chromatography_instruments) > 0:
    print('Chromatography instruments: {}'.format(list(top_mods_ms.chromatography_instruments)))
else:
    print('no chromatography used or no information supplied')

if len(top_mods_ms.instruments) > 0:
    print('Data acquisition instruments: {}'.format(list(top_mods_ms.instruments)))    
if len(top_mods_ms.injection_modes) > 0:
    print('Injection modes: {}'.format(list(top_mods_ms.injection_modes)))
if len(top_mods_ms.acquisition_modes) > 0:
    print('Acquisition modes: {}'.format(list(top_mods_ms.acquisition_modes)))


Then, we add the mass spectrometry assay to the plan:

In [ ]:
try:
    plan.add_assay_type(assay_type_2)
    plan.add_assay_plan_record("blood", assay_type_2)

    assay_plan = next(iter(plan.assay_plan))
except NameError:
    print("There was an assay type left undefined")

And let's see the json representation of the SampleAssayPlan:

In [ ]:
print(json.dumps(plan, 
                 cls=SampleAssayPlanEncoder, 
                 sort_keys=True, 
                 indent=4, 
                 separators=(',', ': ')))

As the last step, we generate assay tables from the information on the Assay Plans and visualise the output files.

In [ ]:
isa_investigation.studies = [isa_object_factory.create_assays_from_plan()]

for assay in isa_investigation.studies[-1].assays:
    print('Assay generated: {0}, {1} samples, {2} processes, {3} data files'
          .format(assay.filename, 
                  len(assay.samples), 
                  len(assay.process_sequence), 
                  len(assay.data_files)))
dataframes = dumpdf(isa_investigation)

In [ ]:
import pandas as pd
table = pd.DataFrame()
try:
    table = dataframes[next(iter(dataframes.keys()))]
except StopIteration:
    print("Need more information to print the table")
    
show_grid(table)

In [ ]:
table = pd.DataFrame()
try:
    table = dataframes['a_blood_ms_FIA_positive_assay.txt']
except KeyError:
    print("Need more details to print the assay table")
show_grid(table)    

In [ ]:
table = pd.DataFrame()
try:
    table = dataframes['a_blood_ms_FIA_negative_assay.txt']
except KeyError:
    print("Need more details to print the assay table")
show_grid(table)    

In [ ]:
table = pd.DataFrame()
try:
    table = dataframes['a_kidney_dnaseq_Illumina HiSeq 2000_assay.txt']
except KeyError:
    print("Need more details to print the assay table")
show_grid(table)    